# Calculate Stokes Number

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv
import seaborn as sb

In [13]:
# Define the directory hosting the solution directories
particles = pv.read('/Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/VTK/lagrangian/kinematicCloudTracks/kinematicCloudTracks_1000.vtk')

In [14]:
def read_mesh(nfile):
    """ Read the OpenFOAM solution, needs the file generated by paraFoam (e.g. *.OpenFOAM) """
    # Read the reference file created by openFOAM `parafoam`
    reader = pv.POpenFOAMReader(nfile)
    # set the active time 
    reader.set_active_time_value(reader.time_values[-1])
    reader.cell_to_point_creation = True
    # Define the mesh
    mesh = reader.read()
    # Define the internal mesh and boundaries
    internal = mesh["internalMesh"]
    boundaries = mesh["boundary"]
    
    return internal, boundaries

In [15]:
def slice_center(mesh, NORMAL='z', ORIGIN=[0, 0, 0], translate=False, CONTOUR=False):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [11]:
mesh, boundaries = read_mesh(solution_dir)

ERROR:root:Cannot open directory: /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/VTK/lagrangian/kinematicCloudTracks/constant/
2023-06-10 21:58:17.363 (  60.498s) [           AE0E1]  vtkOpenFOAMReader.cxx:11303  ERR| vtkPOpenFOAMReader (0x10580b5f0): Cannot open directory: /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/VTK/lagrangian/kinematicCloudTracks/constant/
ERROR:root:Algorithm vtkPOpenFOAMReader (0x10580b5f0) returned failure for request: vtkInformation (0x600003c3d680)
2023-06-10 21:58:17.366 (  60.500s) [           AE0E1]       vtkExecutive.cxx:741    ERR| vtkCompositeDataPipeline (0x60000243c000): Algorithm vtkPOpenFOAMReader (0x10580b5f0) returned failure for request: vtkInformation (0x600003c3d680)
  Debug: Off
  Modified Time: 600
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_INFORMATION
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0




AttributeError: 'NoneType' object has no attribute 'GetNumberOfValues'

In [6]:
mesh

PolyData (0x103d74940)
  N Cells:    736601
  N Points:   736601
  N Strips:   0
  X Bounds:   -8.000e+00, 1.000e+01
  Y Bounds:   -2.260e-01, 3.260e-03
  Z Bounds:   -1.547e-02, 4.090e-01
  N Arrays:   12

In [15]:
boundaries

MultiBlock (0x15ecde8e0)
  N Blocks:	5
  X Bounds:	-150.000, 50.000
  Y Bounds:	-100.000, 100.000
  Z Bounds:	-100.000, 100.000

In [26]:
nslice = slice_center(mesh, NORMAL='z', ORIGIN=[15, 0, 0])

2023-06-10 20:01:20.778 (2075.656s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-10 20:01:20.787 (2075.665s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-10 20:01:21.066 (2075.944s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-10 20:01:21.328 (2076.205s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


In [33]:
# Create the plotter
pt = pv.Plotter()
# Add the slice mesh
pt.add_mesh(nslice, 
            scalars='nut', 
            preference='point', 
        )
pt.show(cpos='xy')

Widget(value="<iframe src='http://localhost:51143/index.html?ui=P_0x177e8c0a0_10&reconnect=auto' style='width:…

## Calculate Reynolds Number

In [38]:
reynolds = (mesh.point_data['rho'] * mesh.point_data['U'][:, 0] * 200) / 1.82E-05

In [39]:
reynolds

pyvista_ndarray([1.3481015e+09, 1.3481016e+09, 1.3481016e+09, ...,
                 1.3481056e+09, 1.3481056e+09, 1.3481056e+09],
                dtype=float32)

In [57]:
stokes = (2/9) * ((1e-4 / 200)**2) * (reynolds)

In [58]:
mesh['Re'] = reynolds
mesh['Stk'] = stokes

In [59]:
mesh

UnstructuredGrid (0x1077d6d00)
  N Cells:    17056335
  N Points:   19785839
  X Bounds:   -1.500e+02, 5.000e+01
  Y Bounds:   -1.000e+02, 1.000e+02
  Z Bounds:   -1.000e+02, 1.000e+02
  N Arrays:   22

In [60]:
nslice = slice_center(mesh, NORMAL='z', ORIGIN=[15, 0, 0])

2023-06-10 20:30:57.695 (3852.559s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-10 20:30:57.704 (3852.569s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-10 20:30:57.981 (3852.845s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-06-10 20:30:58.244 (3853.108s) [           A41E8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


In [61]:
# Create the plotter
pt = pv.Plotter()
# Add the slice mesh
pt.add_mesh(nslice, 
            scalars='Re', 
            preference='point', 
        )
pt.show(cpos='xy')

Widget(value="<iframe src='http://localhost:51143/index.html?ui=P_0x3729c8400_16&reconnect=auto' style='width:…

In [62]:
# Create the plotter
pt = pv.Plotter()
# Add the slice mesh
pt.add_mesh(nslice, 
            scalars='Stk', 
            preference='point', 
        )
pt.show(cpos='xy')

Widget(value="<iframe src='http://localhost:51143/index.html?ui=P_0x3729c8fd0_17&reconnect=auto' style='width:…